# CC3001 Otoño 2020 Tarea 5
# Simulación con Colas y Colas de Prioridad

Profesores: Sección 1 Patricio Poblete / Sección 2 Nelson Baloian

Plazo: Miércoles 24 de junio de 2020 a las 23:59

## Jorge Salvo Marín

El objetivo de esta tarea es aprender a escribir programas de simulación sencillos, haciendo uso de colas y de colas de prioridad.

En primer lugar, consideremos una caja a la que van llegando clientes en instantes aleatorios, separados por intervalos de tamaño promedio $t_a$. Cada cliente que llega se pone al final de la cola, espera su turno, y cuando llega a la cabeza de la cola lo empiezan a atender. La atención dura también un tiempo aleatorio de tamaño promedio $t_s$. Cuando completa su atención, sale del sistema, y en la caja se empieza a atender al siguiente cliente de la cola, en caso de que haya alguien.

![colaT5](https://github.com/ppoblete/Tareas-CC3001-2020-1/blob/master/colaT5.png?raw=1)

Los tiempos entre llegadas y el tiempo de atención se modelan con distribuciones exponenciales, lo cual es la suposición usual en este tipo de simulaciones.

A continuación se ve el programa que simula este proceso para un número dado de clientes y calcula el largo promedio de la cola y la demora total promedio de los clientes, programa el cual fue entregado por los docentes.

In [3]:
import numpy as np
class Cola:
    def __init__(self):
        self.q=[]
    def enq(self,x):
        self.q.insert(0,x)
    def deq(self):
        assert len(self.q)>0
        return self.q.pop()
    def is_empty(self):
        return len(self.q)==0
    def size(self):
        return len(self.q)

def print_list(L):
    for x in L:
        print(x)

def simula(maxclientes,ta,ts,debug=False):
    # simula el paso de maxclientes clientes por el sistema,
    # ta = tiempo promedio entre llegadas (arrivals)
    # ts = tiempo promedio de servicio
    # maxclientes = número maximo de clientes para la simulacion
    # debug = True => genera bitácora y usa secuencia aleatoria reproducible
    # retorna (largo promedio de la cola, tiempo promedio en el sistema)

    if debug:
        np.random.seed(1234)
    ahora=0 # tiempo simulado
    nclientes=0 # número de clientes que han llegado
    c=Cola()
    evento_llegada=np.random.exponential(ta)
    evento_salida=None
    acum_largo_cola=0 # acumula largo de la cola para sacar promedio al final
    acum_tiempo_en_sistema=0 # acumula tiempos en el sistema para sacar promedio al final
    bitacora=[] # En caso que se pida debug
    
    while evento_llegada is not None or evento_salida is not None:
        # Ver si el próximo evento que toca que ocurra es la llegada o la salida de un cliente
        if (evento_salida is None) or (evento_llegada is not None and (evento_llegada<=evento_salida)):
            # Llega un cliente, avanzamos el tiempo simulado a la hora en que llega
            ahora=evento_llegada
            nclientes+=1
            
            if debug:
                bitacora.append("T="+str(ahora)+
                                " evento_llegada="+str(evento_llegada)+" evento_salida="+str(evento_salida)+
                                " => LLEGADA"+
                                "\n\tcola="+str(c.q))
            # El cliente que acaba de llegar se pone a la cola
            c.enq(ahora) # Basta anotar su hora de llegada
            acum_largo_cola+=c.size()
            
            if c.size()==1: # la cola estaba vacía, lo empezamos a atender de inmediato
                evento_salida=ahora+np.random.exponential(ts) # programamos su hora de salida
            
            if nclientes<maxclientes: # programamos la hora de llegada del siguiente cliente
                evento_llegada=ahora+np.random.exponential(ta)
            else:
                evento_llegada=None
        else:               
            # Termina de atenderse un cliente, avanzamos el tiempo simulado a la hora en que sale
            ahora=evento_salida
            if debug:
                bitacora.append("T="+str(ahora)+
                                " evento_llegada="+str(evento_llegada)+" evento_salida="+str(evento_salida)+
                                " => SALIDA"+
                                "\n\tcola="+str(c.q))
            tllegada=c.deq() # sacamos al cliente de la cola
            acum_tiempo_en_sistema+=(ahora-tllegada)
            
            if c.is_empty(): # no hay nadie en la cola
                evento_salida=None
            else: # Hay un cliente esperando, empezamos a atenderlo
                evento_salida=ahora+np.random.exponential(ts) # programamos su hora de salida
    
    return (acum_largo_cola/maxclientes,acum_tiempo_en_sistema/maxclientes,bitacora)

In [4]:
(cola,demora,bitacora)=simula(10,100,75,debug=True)
print("Largo promedio de la cola=",cola," Tiempo promedio en el sistema=",demora)

Largo promedio de la cola= 1.9  Tiempo promedio en el sistema= 138.02762271192722


In [5]:
print_list(bitacora)

T=21.25986576184801 evento_llegada=21.25986576184801 evento_salida=None => LLEGADA
	cola=[]
T=78.83677538497456 evento_llegada=78.83677538497456 evento_salida=94.24603167524747 => LLEGADA
	cola=[21.25986576184801]
T=94.24603167524747 evento_llegada=232.71542283099382 evento_salida=94.24603167524747 => SALIDA
	cola=[78.83677538497456, 21.25986576184801]
T=207.7973648436669 evento_llegada=232.71542283099382 evento_salida=207.7973648436669 => SALIDA
	cola=[78.83677538497456]
T=232.71542283099382 evento_llegada=232.71542283099382 evento_salida=None => LLEGADA
	cola=[]
T=256.5855663702635 evento_llegada=265.07595569718336 evento_salida=256.5855663702635 => SALIDA
	cola=[232.71542283099382]
T=265.07595569718336 evento_llegada=265.07595569718336 evento_salida=None => LLEGADA
	cola=[]
T=386.489172340086 evento_llegada=582.4168679105603 evento_salida=386.489172340086 => SALIDA
	cola=[265.07595569718336]
T=582.4168679105603 evento_llegada=582.4168679105603 evento_salida=None => LLEGADA
	cola=[]


## Mejora

### 1. Introducir cola de prioridad

El programa está escrito sabiendo que puede haber dos tipos de eventos por ocurrir (LLEGADA y SALIDA) y que en todo momento puede haber a lo más solo uno de cada tipo.

En una situación más general, podría haber eventos de muchos tipos programados para ocurrir, en muchos instantes distintos. En ese caso, la implementación presentada es demasiado limitada, y se requiere un mecanismo más general para administrar esa lista de eventos futuros. Una *cola de prioridad* (usando el tiempo como prioridad, y en que el mínimo tiempo equivale a mejor prioridad) es la estructura adecuada, porque en esa lista de eventos futuros siempre vamos extrayendo el primero que debe ocurrir (el mínimo), y durante la ejecución se pueden generar nuevos elementos programados para ocurrir en el futuro (inserciones).

En preparación a poder simular modelos más complejos, lo que usted tiene que hacer es implementar una lista de eventos futuros mediante uan cola de prioridad, en la cual se almacenen pares de la forma

$$
(\text{tiempo},\text{tipo de evento})
$$

y modificar el programa para que utilice esa cola de prioridad en lugar de las variables ``evento_salida`` y ``evento_llegada``. Implemente la cola de prioridad en base al código para heaps que aparece en el apunte.

El código que usaremos será el del Heap,, que se puede encontrar en cátedra, pero con una pequeña modificación, el Heap de cátdra recibe valores int, pero el que se necesita es uno qe reciba un objeto, con un cierto evento (entrada o salida) y el tiempo aleatorio que se genera para dicho evento. A esta clase objeto la definiremos coo Persona, la cual contiene el tiempo con self.time y el tipo de evento con self.eve.

Si bien no se cambiará para que la mejor prioridad sea el menor tiempo, se agregará la función de extraer el mínimo valor de Heap.

In [14]:
import numpy as np
def print_list(L):
    for x in L:
        print(x)

class Persona:
    def __init__(self, time, eve):
        self.time=time
        self.eve=eve


def trepar(a,j): # El elemento a[j] trepa hasta su nivel de prioridad 
    while j>=1 and a[j].time > a[(j-1)//2].time:
        (a[j], a[(j-1)//2]) = (a[(j-1)//2], a[j]) # intercambiamos con el padre
        j=(j-1)//2 # subimos al lugar del padre

def hundir(a,j,n): # El elemento a[j] se hunde hasta su nivel de prioridad
    while 2*j+1<n: # mientras tenga al menos 1 hijo
        k=2*j+1 # el hijo izquierdo
        if k+1<n and a[k+1].time > a[k].time: # el hijo derecho existe y es mayor
            k+=1
        if a[j].time >= a[k].time: # tiene mejor prioridad que ambos hijos
            break
        (a[j],a[k])=(a[k],a[j]) # se intercambia con el mayor de los hijos
        j=k # bajamos al lugar del mayor de los hijos

class Heap:
    def __init__(self,maxn=100):
        self.a=np.zeros(maxn,dtype='object')
        self.n=0
    def insert(self,x):
        assert self.n<len(self.a)
        self.a[self.n]=x    
        trepar(self.a,self.n)
        self.n+=1       
    def extract_max(self):
        x=self.a[0] # esta variable lleva el máximo, el casillero 0 queda vacante
        self.n-=1   # achicamos el heap
        self.a[0]=self.a[self.n] # movemos el elemento sobrante hacia el casillero vacante
        hundir(self.a,0,self.n)
        return x
    def extract_min(self):
        self.n -=1
    def mostrar_ult(self):
        ult = self.a[self.n-1]
        return ult


def simula(maxclientes,ta,ts,debug=False):
    # ta = tiempo promedio entre llegadas (arrivals)
    # ts = tiempo promedio de servicio
    # maxclientes = número maximo de clientes para la simulacion
    # debug = True => genera bitácora y usa secuencia aleatoria reproducible
    # retorna (largo promedio de la cola, tiempo promedio en el sistema)
    if debug:
        np.random.seed(1234)
    h=Heap()        # Creamos el Heap, originalmente vacío
    p = Persona(np.random.exponential(ta), 'entrada')  #Asignamos un objeto con un tiempo de entrada    
    h.insert(p)     # insertamos el objeto en el heap como el primero que entra
    nclientes = 1   # número de clientes que han llegado
    ahora = p.time  # tiempo simulado, empieza cuando llega la primera persona
    largo_cola = 1  # largo actual de la cola
    acum_largo_cola = 1      # acumula largo de la cola para sacar promedio al final
    acum_tiempo_en_sistema=0 # acumula tiempos en el sistema para sacar promedio al final
    bitacora=[]              # En caso que se pida debug
    if debug:
        bitacora.append("T="+ str(ahora) + " => LLEGADA, " + 'Largo de cola=' + str(largo_cola))
    
    while largo_cola > 0:        # mientras hayan persona en la cola
        ult = h.mostrar_ult()    
        while type(ult) != int and ult.eve == 'salida':  # agregamos que el ultimo sea distinto de int ya que
                                                         # tiraba error cuano quedaba el heap vacío
            h.extract_min()
            largo_cola -= 1 
            if debug:
                bitacora.append("T="+ str(ult.time) + " => SALIDA, " + 'Largo de cola=' + str(largo_cola))
            ult = h.mostrar_ult()
        if type(ult) != int and ult.eve == 'entrada':
            h.extract_min()
            p = Persona(ult.time + np.random.exponential(ts), 'salida')
            acum_tiempo_en_sistema += p.time - ult.time
            h.insert(p)
        if type(ult) != int and nclientes < maxclientes:
            p = Persona(ahora + np.random.exponential(ta), 'entrada')
            nclientes+=1
            largo_cola += 1 
            h.insert(p)
            ahora = p.time
            if debug:
                bitacora.append("T="+ str(ahora) + " => LLEGADA, " + 'Largo de cola=' + str(largo_cola))      
        acum_largo_cola += largo_cola
            
    return (acum_largo_cola/maxclientes,acum_tiempo_en_sistema/maxclientes,bitacora)

In [15]:
(cola,demora,bitacora)=simula(10,100,75,debug=True)
print("Largo promedio de la cola=",cola," Tiempo promedio en el sistema=",demora)

Largo promedio de la cola= 2.3  Tiempo promedio en el sistema= 85.93828845920461


In [16]:
print_list(bitacora)

T=21.25986576184801 => LLEGADA, Largo de cola=1
T=78.83677538497456 => LLEGADA, Largo de cola=2
T=230.23855294286716 => LLEGADA, Largo de cola=3
T=194.245760969489 => SALIDA, Largo de cola=2
T=94.24603167524747 => SALIDA, Largo de cola=1
T=262.5990858090567 => LLEGADA, Largo de cola=2
T=254.10869648213688 => SALIDA, Largo de cola=1
T=579.9399980224337 => LLEGADA, Largo de cola=2
T=384.01230245195933 => SALIDA, Largo de cola=1
T=624.2282369781022 => LLEGADA, Largo de cola=2
T=739.2597305574725 => LLEGADA, Largo de cola=3
T=736.4597924738038 => SALIDA, Largo de cola=2
T=676.3636930872223 => SALIDA, Largo de cola=1
T=785.503086788047 => LLEGADA, Largo de cola=2
T=855.4363469391337 => LLEGADA, Largo de cola=3
T=847.2808012041539 => SALIDA, Largo de cola=2
T=832.8023831803557 => SALIDA, Largo de cola=1
T=1003.6405240981396 => LLEGADA, Largo de cola=2
T=856.4761554746091 => SALIDA, Largo de cola=1
T=1164.329470875144 => SALIDA, Largo de cola=0


## 2. Introducir un tiempo comprando

A continuación, introduciremos un elemento adicional en el modelo. Supondremos que una vez que un cliente llega, en lugar de ponerse a la cola de inmediato, pasa un rato en la tienda comprando, y una vez que termina (después de transcurrido un tiempo aleatorio de duración promedio $t_c$), en ese momento recién se pone a la cola para pagar en la caja.

![cola2T5](https://github.com/ppoblete/Tareas-CC3001-2020-1/blob/master/cola2T5.png?raw=1)

Observe que, como puede haber muchos clientes en la tienda que aún no terminan de comprar, en la lista de eventos futuros ahora puede haber muchos eventos de un nuevo tipo (FINCOMPRA), lo que justifica el haber introducido la cola de prioridad.

Ejecutamos el programa en modo ``debug`` agregando un parámetro $t_c=300$ e imprimimos el largo promedio de la cola, la demora total promedio de los clientes y la bitácora.

Para esta implementación no se requiere un cambio en el Heap ni la clase Persona, solo una pequeña modificacion en simula, agregando una variable de entrada tc correpondiente al promedio de tiempo en comprar, entonces cuando se le asigna una tiempo de entrada a cierta persona, luego se le debe asignar un tiempo de compra y luego un tiempo de salida, agragando un if y una sección a a bitácora commo se aprecia a continuación.

In [11]:
import numpy as np
def print_list(L):
    for x in L:
        print(x)
        
class Persona:
    def __init__(self, time, eve):
        self.time=time
        self.eve=eve


def trepar(a,j): # El elemento a[j] trepa hasta su nivel de prioridad 
    while j>=1 and a[j].time > a[(j-1)//2].time:
        (a[j], a[(j-1)//2]) = (a[(j-1)//2], a[j]) # intercambiamos con el padre
        j=(j-1)//2 # subimos al lugar del padre

def hundir(a,j,n): # El elemento a[j] se hunde hasta su nivel de prioridad
    while 2*j+1<n: # mientras tenga al menos 1 hijo
        k=2*j+1 # el hijo izquierdo
        if k+1<n and a[k+1].time > a[k].time: # el hijo derecho existe y es mayor
            k+=1
        if a[j].time >= a[k].time: # tiene mejor prioridad que ambos hijos
            break
        (a[j],a[k])=(a[k],a[j]) # se intercambia con el mayor de los hijos
        j=k # bajamos al lugar del mayor de los hijos

class Heap:
    def __init__(self,maxn=100):
        self.a=np.zeros(maxn,dtype='object')
        self.n=0
    def insert(self,x):
        assert self.n<len(self.a)
        self.a[self.n]=x    
        trepar(self.a,self.n)
        self.n+=1       
    def extract_max(self):
        x=self.a[0] # esta variable lleva el máximo, el casillero 0 queda vacante
        self.n-=1   # achicamos el heap
        self.a[0]=self.a[self.n] # movemos el elemento sobrante hacia el casillero vacante
        hundir(self.a,0,self.n)
        return x
    def extract_min(self):
        self.n -=1
    def mostrar_ult(self):
        ult = self.a[self.n-1]
        return ult

def simula(maxclientes,ta,ts,tc,debug=False):
    # ta = tiempo promedio entre llegadas (arrivals)
    # ts = tiempo promedio de servicio
    # tc = tiempo promedio de compra
    # maxclientes = número maximo de clientes para la simulacion
    # debug = True => genera bitácora y usa secuencia aleatoria reproducible
    # retorna (largo promedio de la cola, tiempo promedio en el sistema)
    if debug:
        np.random.seed(1234)
    h=Heap()        # Creamos el Heap, originalmente vacío
    p = Persona(np.random.exponential(ta), 'entrada')  #Asignamos un objeto con un tiempo de entrada    
    h.insert(p)     # insertamos el objeto en el heap como el primero que entra
    nclientes = 1   # número de clientes que han llegado
    ahora = p.time  # tiempo simulado, empieza cuando llega la primera persona
    largo_cola = 0  # largo actual de la cola
    acum_largo_cola = 1      # acumula largo de la cola para sacar promedio al final
    acum_tiempo_en_sistema=0 # acumula tiempos en el sistema para sacar promedio al final
    bitacora=[]              # En caso que se pida debug
    if debug:
        bitacora.append("T="+ str(ahora) + " => LLEGADA, " + 'Largo de cola=' + str(largo_cola))
    
    while largo_cola >= 0:        # mientras hayan persona en la cola
        ult = h.mostrar_ult()
        while type(ult) != int and ult.eve == 'salida':  
            h.extract_min()
            largo_cola -= 1
            if debug:
                bitacora.append("T="+ str(ult.time) + " => SALIDA, " + 'Largo de cola=' + str(largo_cola))
            ult = h.mostrar_ult() 
        if type(ult) != int and ult.eve == 'entrada':
            h.extract_min()
            p = Persona(ult.time + np.random.exponential(tc), 'cola')
            h.insert(p)
            largo_cola += 1
            acum_tiempo_en_sistema += p.time - ult.time
            if debug:
                bitacora.append("T="+ str(p.time) + " => A LA COLA, " + 'Largo de cola=' + str(largo_cola))  
        if type(ult) != int and ult.eve == 'cola':
            h.extract_min()
            p = Persona(ult.time + np.random.exponential(ts), 'salida')
            acum_tiempo_en_sistema += p.time - ult.time
            h.insert(p)
        if type(ult) != int and nclientes < maxclientes:
            p = Persona(ahora + np.random.exponential(ta), 'entrada')
            nclientes+=1
            h.insert(p)
            ahora = p.time
            if debug:
                bitacora.append("T="+ str(ahora) + " => LLEGADA, " + 'Largo de cola=' + str(largo_cola))      
        acum_largo_cola += largo_cola
        
        if largo_cola == 0:
            return (acum_largo_cola/maxclientes, acum_tiempo_en_sistema/maxclientes, bitacora)

In [12]:
(cola,demora,bitacora)=simula(10,100,75,300,debug=True)
print("Largo promedio de la cola=",cola," Tiempo promedio en el sistema=",demora)

Largo promedio de la cola= 8.8  Tiempo promedio en el sistema= 361.99239759717227


In [13]:
print_list(bitacora)

T=21.25986576184801 => LLEGADA, Largo de cola=0
T=313.20452941544585 => A LA COLA, Largo de cola=1
T=78.83677538497456 => LLEGADA, Largo de cola=1
T=540.4727177230324 => A LA COLA, Largo de cola=2
T=230.23855294286716 => LLEGADA, Largo de cola=2
T=325.719127099946 => A LA COLA, Largo de cola=3
T=262.5990858090567 => LLEGADA, Largo de cola=3
T=748.2519523806671 => A LA COLA, Largo de cola=4
T=579.9399980224337 => LLEGADA, Largo de cola=4
T=624.2282369781022 => LLEGADA, Largo de cola=4
T=739.2597305574725 => LLEGADA, Largo de cola=4
T=998.3988474696353 => A LA COLA, Largo de cola=5
T=785.503086788047 => LLEGADA, Largo de cola=5
T=855.4363469391337 => LLEGADA, Largo de cola=5
T=696.9925121744025 => SALIDA, Largo de cola=4
T=374.9822438315528 => SALIDA, Largo de cola=3
T=743.4189646993739 => A LA COLA, Largo de cola=4
T=1003.6405240981396 => LLEGADA, Largo de cola=4
T=1428.2588738960642 => A LA COLA, Largo de cola=5
T=991.6215699279084 => A LA COLA, Largo de cola=6
T=1141.6916868470114 => 

Como se observa en el programa panteado, la implementación de la cola de prioridad acota el código, reduciendo la cantidad de variables guardadas y permite agregarle, sin gran esfuerzo, nuevas características lo cual habla de la eficiencia de la implementación.